In [103]:
import pandas as pd

In [104]:
historical_figures_list = pd.read_excel(r'Historical Figures List.xlsx')

figures = sorted(list(historical_figures_list.iloc[:,1].values))

figures_joined = ', '.join(figures)

In [112]:
import json
import openai
import os
from openai._client import OpenAI

# Initialise OpenAI and create the function to get the results 
os.environ["OPENAI_API_KEY"] = json.load(open("Credentials.json"))['OPENAI_API_KEY']
client = OpenAI()
    
llm_model="gpt-3.5-turbo"

def get_completion(prompt, model=llm_model):
    messages = [{"role": "user", "content": prompt}]
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.9, 
    )
    return response.choices[0].message.content

In [113]:
prompt = """Please generate a list of influential but lesser-known historical figures from various fields such as science, arts, politics, and exploration. These figures should be individuals who are not commonly cited in mainstream historical narratives but have made significant contributions in their respective areas.

For each figure, provide a brief description of their contributions and historical significance. The output should be formatted as a JSON array, with each element being an object that contains 'Name' and 'Description' keys. Ensure that the figures are diverse in terms of their time period, geographical origin, and field of influence. Here is the desired format:

[
    {"Name": "Unique Historical Figure", "Description": "A brief description of their contributions and significance"},
    ...
]

I am looking for a minimum of 20 figures. It is crucial that these figures are not well-known names and have not been commonly featured in standard historical accounts or lists.
"""

In [114]:
new_figures = get_completion(prompt)

resp_dict = json.loads(new_figures2)

df = pd.DataFrame(resp_dict)

In [115]:
filtered_df = df[~df['Name'].isin(figures)].copy()

# We want to add the figure_id
max_figure_id = max(historical_figures_list['Figure_ID'])

filtered_df['Figure_ID'] = range(max_figure_id + 1, max_figure_id + 1 + len(filtered_df))
filtered_df['Script_Created'] = "No"
filtered_df['AI_Voice_Generated'] = "No"
filtered_df['Images_Obtained'] = "No"
filtered_df['Youtube_Video_Created'] = "No"
filtered_df['Youtube_Video_Posted'] = "No"
filtered_df['Youtube_URL'] = None
filtered_df['Prompt'] = None

column_to_move = 'Figure_ID'
new_columns = [column_to_move] + [col for col in filtered_df.columns if col != column_to_move]

# Reorder the DataFrame
filtered_df = filtered_df[new_columns]

In [116]:
filtered_df

,Figure_ID,Name,Description,Script_Created,AI_Voice_Generated,Images_Obtained,Youtube_Video_Created,Youtube_Video_Posted,Youtube_URL,Prompt


In [117]:
# Appending filtered_df to historical_figures_list
combined_df = pd.concat([historical_figures_list, filtered_df], ignore_index=True)

In [118]:
# Once all done stop the engine and then overwrite the Excel file
combined_df.to_excel('Historical Figures List.xlsx', index=False)